In [ ]:
import gensim
import gensim.downloader
import spacy
import nltk
import random
import numpy as np

import clustering_class
from clustering_class import HierarchicalClustering

random.seed(1)

In [ ]:
# import vectors & the nlp word interpreter
glove_vectors = gensim.downloader.load("glove-wiki-gigaword-50")
# nlp = spacy.load('en_core_web_trf')

In [ ]:
from nltk.corpus import brown

len(brown.words())

In [ ]:
vocab_size = 5000
tagged_vocab = nltk.pos_tag(glove_vectors.index_to_key[:vocab_size])

verbs = [verb[0] for verb in tagged_vocab if verb[1] == "VB"]
nouns = [noun[0] for noun in tagged_vocab if noun[1] == "NN"]
# print(nouns)

print(f"There are {len(verbs)} verbs and {len(nouns)} nouns.")

verb_indices = [glove_vectors.index_to_key.index(verb) for verb in verbs]
noun_indices = [glove_vectors.index_to_key.index(noun) for noun in nouns][2:]

chosen_indices = set(noun_indices)

In [ ]:
# trying to use wordnet instead
from nltk.corpus import wordnet as wn

wordnet_chosen_words = []
for word in glove_vectors.index_to_key[90:5000]:
    tmp = wn.synsets(word)
    if len([t.pos() for t in tmp if t.pos() == "n"]) >= 1:
        wordnet_chosen_words.append(word)

wordnet_chosen_indices = set(
    [glove_vectors.key_to_index[word] for word in wordnet_chosen_words]
)
print(len(wordnet_chosen_indices))
# print(wordnet_chosen_words)

In [ ]:
# re-import the clustering class in case it was changed.
import importlib

importlib.reload(clustering_class)

# constants
proximity_const = 1
reducing_coef_const = 0
increasing_proximity_const = 0.2

# normed vectors
reduced_vectors = glove_vectors.get_normed_vectors()[:vocab_size]

hierarchical_clustering = clustering_class.HierarchicalClustering(
    word_embedding=glove_vectors,
    list_of_vectors=reduced_vectors,
    chosen_indices=wordnet_chosen_indices,
    initial_proximity=proximity_const,
    proximity_reduc=reducing_coef_const,
    initial_proximity_inc=increasing_proximity_const,
    verbose=True,
)


(
    better_hier_list,
    better_hier_list_w,
) = hierarchical_clustering.get_better_list_of_hierarchical_orders()

# try to give it the similarity measure instead?

# named entity recognition, filter out
# visualization of hierarchy

In [ ]:
chosen_words = np.asarray([reduced_vectors[i] for i in wordnet_chosen_indices])

In [ ]:
hierarchical_clustering.visualize_vectors(chosen_words)

In [ ]:
i = 5

print(len(better_hier_list[i].keys()))
better_hier_list_w[i].keys()

In [ ]:
glove_vectors.key_to_index["philosophy"]
print(glove_vectors.most_similar("philosophy"))
print(len(hierarchical_clustering.which_in_proximity(0, 1)))
for key in hierarchical_clustering.which_in_proximity(4044, 1):
    print(glove_vectors.index_to_key[key])

So what would be a good measure of being a relevant word in a corpus of text. 

In [ ]:
glove_vectors.vectors.shape